## A tour of MLJ

### Models, machines, basic training and testing

Let's load data and define train and test rows:

In [2]:
using MLJ
using DataFrames, Statistics

Xraw = rand(300,3)
y = exp(Xraw[:,1] - Xraw[:,2] - 2Xraw[:,3] + 0.1*rand(300))
X = DataFrame(Xraw)

train, test = partition(eachindex(y), 0.70); # 70:30 split

A *model* is a container for hyperparameters:

In [3]:
knn_model=KNNRegressor(K=10)

# KNNRegressor{Float64} @ 1…89: 
target_type             =>   Float64
K                       =>   10
metric                  =>   euclidean (generic function with 1 method)
kernel                  =>   reciprocal (generic function with 1 method)



Wrapping the model in data creates a *machine* which will store training outcomes (called *fit-results*):

In [4]:
knn = machine(knn_model, X, y)

# Machine{KNNRegressor{Float64}} @ 1…86: 
model                   =>   KNNRegressor{Float64} @ 1…89
fitresult               =>   (undefined)
cache                   =>   (undefined)
args                    =>   (omitted Tuple{DataFrame,Array{Float64,1}} of length 2)
report                  =>   (undefined)
rows                    =>   (undefined)



Training on the training rows and evaluating on the test rows:

In [5]:
fit!(knn, rows=train)
yhat = predict(knn, X[test,:])
rms(y[test], yhat)

┌ Info: Training Machine{KNNRegressor{Float64}} @ 1…86.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/machines.jl:93


0.09078144921694162

Or, we could have skipped the train/test definitions and evaluated one line:

In [6]:
evaluate!(knn, resampling=Holdout(fraction_train=0.7), measure=rms)

0.09078144921694162

Our machine/model constructions and associateed fit/predict syntax anticipates a powerful extension for building networks of learners described later. Changing a hyperparameter and re-evaluating:

In [7]:
knn_model.K = 20
evaluate!(knn, resampling=Holdout(fraction_train=0.7))  # `default_measure(knn) == rms` so `measure` kwarg can be dropped

0.11375481529099939

### Homogeneous ensembles

Here's a bagged ensemble model for 20 K-nearest neighbour regressors:

In [8]:
ensemble_model = EnsembleModel(atom=knn_model, n=20)

# DeterministicEnsembleModel @ 1…12: 
atom                    =>   KNNRegressor{Float64} @ 1…89
weights                 =>   0-element Array{Float64,1}
bagging_fraction        =>   0.8
rng_seed                =>   0
n                       =>   20
parallel                =>   true



In [9]:
@more

# DeterministicEnsembleModel @ 1…12: 
atom                    =>   KNNRegressor{Float64} @ 1…89
weights                 =>   0-element Array{Float64,1}
bagging_fraction        =>   0.8
rng_seed                =>   0
n                       =>   20
parallel                =>   true

## KNNRegressor{Float64} @ 1…89: 
target_type             =>   Float64
K                       =>   20
metric                  =>   euclidean (generic function with 1 method)
kernel                  =>   reciprocal (generic function with 1 method)



It can be trained and tested the same as any other model:

In [10]:
ensemble = machine(ensemble_model, X, y)
estimates = evaluate!(ensemble, resampling=CV())

Cross-validating: 100%[=========================] Time: 0:00:06


6-element Array{Float64,1}:
 0.11224997911116968
 0.08307949451639975
 0.04953104214674491
 0.16412111968261445
 0.06412673500541309
 0.14816801971581123

In [11]:
(mean(estimates), std(estimates))

(0.10354606502969217, 0.04610119519434663)

### Systematic tuning

Let's simultaneously tune the ensemble's `bagging_fraction` and the K-nearest neighbour hyperparameter `K`. Since one of our models is a field of the other, we have nested hyperparameters:

In [12]:
params(ensemble_model)

(atom = (target_type = Float64, K = 20, metric = MLJ.KNN.euclidean, kernel = MLJ.KNN.reciprocal), weights = Float64[], bagging_fraction = 0.8, rng_seed = 0, n = 20, parallel = true)

To define a tuning grid, we construct ranges for the two parameters and collate these ranges following the same pattern above (omitting parameters that don't change):

In [13]:
B_range = range(ensemble_model, :bagging_fraction, lower= 0.5, upper=1.0, scale = :linear)
K_range = range(knn_model, :K, lower=1, upper=100, scale=:log10)
nested_ranges = (atom = (K = K_range,), bagging_fraction = B_range)

(atom = (K = NumericRange @ 1…30,), bagging_fraction = NumericRange @ 1…09)

Now we choose a tuning strategy:

In [14]:
tuning = Grid(resolution=12)

# Grid @ 4…12: 
resolution              =>   12
parallel                =>   true



And a resampling strategy:

In [15]:
resampling = Holdout(fraction_train=0.8)

# Holdout @ 3…32: 
fraction_train          =>   0.8
shuffle                 =>   false



And define a new model which wraps the these strategies around our ensemble model:

In [16]:
tuned_ensemble_model = TunedModel(model=ensemble_model, 
    tuning=tuning, resampling=resampling, nested_ranges=nested_ranges)

# DeterministicTunedModel @ 1…86: 
model                   =>   DeterministicEnsembleModel @ 1…12
tuning                  =>   Grid @ 4…12
resampling              =>   Holdout @ 3…32
measure                 =>   nothing
operation               =>   predict (generic function with 19 methods)
nested_ranges           =>   (omitted NamedTuple{(:atom, :bagging_fraction),Tuple{NamedTuple{(:K,),Tuple{MLJ.NumericRange{Int64,Symbol}}},MLJ.NumericRange{Float64,Symbol}}})
full_report             =>   true



Fitting the corresponding machine tunes the underlying model (in this case an ensemble) and retrains on all supplied data:

In [17]:
tuned_ensemble = machine(tuned_ensemble_model, X[train,:], y[train])
fit!(tuned_ensemble);

┌ Warning: No measure specified. Using measure=MLJ.rms. 
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/machines.jl:82
┌ Info: Training Machine{MLJ.DeterministicTunedMo…} @ 1…23.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/machines.jl:93
Searching a 132-point grid for best model: 100%[=========================] Time: 0:00:07
┌ Info: Training best model on all supplied data.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/tuning.jl:142


For each fitted machine, one may access the fitted parameters (as opposed to the hyperparameters stored in its model). In the current case the "fitted parameter" is the best ensemble model (trained on all available data):

In [18]:
fp = fitted_params(tuned_ensemble)

(best_model = DeterministicEnsembleModel @ 1…20,)

In [19]:
@show fp.best_model.bagging_fraction
@show fp.best_model.atom.K;

(fp.best_model).bagging_fraction = 0.5909090909090909
((fp.best_model).atom).K = 1


The `report` method gives more detail on the tuning process:

In [20]:
r=report(tuned_ensemble)           # named tuple
zip(keys(r), values(r)) |> collect # for better viewing

5-element Array{Tuple{Symbol,Any},1}:
 (:parameter_names, ["atom.K" "bagging_fraction"])                                                                                                                                                                            
 (:parameter_scales, Symbol[:log10 :linear])                                                                                                                                                                                  
 (:parameter_values, Any[1 0.5; 2 0.5; … ; 66 1.0; 100 1.0])                                                                                                                                                                  
 (:measurements, [0.0896036, 0.115431, 0.128693, 0.154714, 0.196045, 0.222209, 0.246999, 0.273775, 0.31375, 0.358156  …  0.0999239, 0.105842, 0.111224, 0.129869, 0.151254, 0.194264, 0.224528, 0.254061, 0.287962, 0.327163])
 (:best_measurement, 0.0786385513959987)                              

Evaluating the tuned model:

In [21]:
yhat = predict(tuned_ensemble, X[test,:])
rms(yhat, y[test])

0.08409339267106415

Or, using all the data, get cross-validation estimates, with cv-tuning on each fold complement (nested resampling):

In [22]:
tuned_ensemble = machine(tuned_ensemble_model, X, y)
evaluate!(tuned_ensemble, resampling=CV(nfolds=4), verbosity=2)

Cross-validating:  20%[=====>                   ]  ETA: 0:00:00┌ Info: Training Machine{MLJ.DeterministicTunedMo…} @ 1…48.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/machines.jl:93
Searching a 132-point grid for best model: 100%[=========================] Time: 0:00:06
┌ Info: Training best model on all supplied data.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/tuning.jl:142
Cross-validating:  40%[==========>              ]  ETA: 0:00:09┌ Info: Training Machine{MLJ.DeterministicTunedMo…} @ 1…48.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/machines.jl:93
Searching a 132-point grid for best model: 100%[=========================] Time: 0:00:06
┌ Info: Training best model on all supplied data.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/tuning.jl:142
Cross-validating:  60%[===============>         ]  ETA: 0:00:08┌ Info: Training Machine{MLJ.DeterministicTunedMo…} @ 1…48.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/machines.jl:93
Searching a 132-point grid for best model: 100%[==

4-element Array{Float64,1}:
 0.08096082109810439
 0.07800556218551255
 0.07327732650564481
 0.08309441293645088

### Learning networks

MLJ has a flexible interface for building networks from multiple machine learning elements, whose complexity extend beyond linear "pipelines", and with a minimal of added abstraction.

In MLJ, a *learning network* is a graph whose nodes apply an operation, such as `predict` or `transform`, using a fixed machine (requiring training) - or which, alternatively, applies a regular (untrained) mathematical operation to its input(s). In practice, a learning network works with *fixed* sources for its training/evaluation data, but can be built and tested in stages. By contrast, an *exported learning network* is a learning network exported as a stand-alone, re-usable `Model` object, to which all the MLJ `Model`  meta-algorthims can be applied (ensembling, systematic tuning, etc). 

As we shall see, exporting a learning network as a reusable model, is very easy. 

### Building a simple learning network

![](wrapped_ridge.png)

The diagram above depicts a learning network which standardises the input data, `X`, learns an optimal Box-Cox transformation for the target, `y`, predicts new targets using ridge regression, and then inverse-transforms those predictions (for later comparison with the original test data). The machines are labelled yellow. 

To implement the network, we begin by loading all data needed for training and evaluation into *source nodes*:

In [23]:
Xs = source(X)
ys = source(y)

Source @ 1…99

We label nodes according to their outputs in the diagram. Notice that the nodes `z` and `yhat` use the same machine `box` for different operations. 

To construct the `W` node we first need to define the machine `stand` that it will use to transform inputs. 

In [24]:
stand_model = Standardizer()
stand = machine(stand_model, Xs)

NodalMachine @ 1…82 = machine(Standardizer @ 1…00, 8…05)

Because `Xs` is a node, instead of concrete data, we can call `transform` on the machine without first training it, and the result is the new node `W`, instead of concrete transformed data:

In [25]:
W = transform(stand, Xs)

Node @ 1…46 = transform(1…82, 8…05)

To get actual transformed data we *call* the node appropriately, which will require we first train the node. Training a node, rather than a machine, triggers training of *all* necessary machines in the network.

In [26]:
fit!(W, rows=train)
W()          # transform all data
W(rows=test) # transform only test data
W(X[3:4,:])  # transform any data, new or old

┌ Info: Training NodalMachine{Standardizer} @ 1…82.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/machines.jl:93


(x1 = [0.376715, 0.561488], x2 = [-0.213696, -0.555213], x3 = [-0.787951, 1.2842])

If you like, you can think of `W` (and the other nodes we will define) as "dynamic data": `W` is *data*, in the sense that  it an be called ("indexed") on rows, but *dynamic*, in the sense the result depends on the outcome of training events. 

The other nodes of our network are defined similarly:

In [27]:
box_model = UnivariateBoxCoxTransformer()  # for making data look normally-distributed
box = machine(box_model, ys)
z = transform(box, ys)

ridge_model = RidgeRegressor(lambda=0.1)
ridge =machine(ridge_model, W, z)
zhat = predict(ridge, W)

yhat = inverse_transform(box, zhat)

Node @ 1…00 = inverse_transform(2…26, predict(5…34, transform(1…82, 8…05)))

We are ready to train and evaluate the completed network. Notice that the standardizer, `stand`, is *not* retrained, as MLJ remembers that it was trained earlier:

In [28]:
fit!(yhat, rows=train)
rms(y[test], yhat(rows=test)) # evaluate

┌ Info: Not retraining NodalMachine{Standardizer} @ 1…82. It is up-to-date.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/networks.jl:201
┌ Info: Training NodalMachine{UnivariateBoxCoxTransfor…} @ 2…26.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/machines.jl:93
┌ Info: Training NodalMachine{RidgeRegressor{Float64}} @ 5…34.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/machines.jl:93


0.020077532519835517

In [29]:
yhat(X[3:4,:])  # predict on new or old data

2-element Array{Float64,1}:
 0.6976639470833352
 0.244671334466375 

We can change hyperparameters and retrain:

In [30]:
ridge_model.lambda = 0.01
fit!(yhat, rows=train) 
rms(y[test], yhat(rows=test))

┌ Info: Not retraining NodalMachine{UnivariateBoxCoxTransfor…} @ 2…26. It is up-to-date.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/networks.jl:201
┌ Info: Not retraining NodalMachine{Standardizer} @ 1…82. It is up-to-date.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/networks.jl:201
┌ Info: Updating NodalMachine{RidgeRegressor{Float64}} @ 5…34.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/machines.jl:97


0.02008343668851138

> **Notable feature.** The machine, `ridge::NodalMachine{RidgeRegressor}`, is retrained, because its underlying model has been mutated. However, since the outcome of this training has no effect on the training inputs of the machines `stand` and `box`, these transformers are left untouched. (During construction, each node and machine in a learning network determines and records all machines on which it depends.) This behaviour, which extends to exported learning networks, means we can tune our wrapped regressor without re-computing transformations each time the hyperparameter is changed. 

### Exporting a learning network as a composite model

To export a learning network:
- Define a new `mutable struct` model type.
- Wrap the learning network code in a model `fit` method.

All learning networks that make determinisic (or, probabilistic) predictions export as models of subtype `Deterministic{Node}` (respectively, `Probabilistic{Node}`):


In [31]:
mutable struct WrappedRidge <: Deterministic{Node}
    ridge_model
end

Now satisfied that our wrapped Ridge Regression learning network works, we simply cut and paste its defining code into a `fit` method: 

In [32]:
function MLJ.fit(model::WrappedRidge, X, y)
    Xs = source(X)
    ys = source(y)

    stand_model = Standardizer()
    stand = machine(stand_model, Xs)
    W = transform(stand, Xs)

    box_model = UnivariateBoxCoxTransformer()  # for making data look normally-distributed
    box = machine(box_model, ys)
    z = transform(box, ys)

    ridge_model = model.ridge_model ###
    ridge =machine(ridge_model, W, z)
    zhat = predict(ridge, W)

    yhat = inverse_transform(box, zhat)
    fit!(yhat, verbosity=0)
    
    return yhat
end

The line marked `###`, where the new exported model's hyperparameter `ridge_model` is spliced into the network, is the only modification.

This completes the export process.

> **What's going on here?** MLJ's machine interface is built atop a more primitive *[model](adding_new_models.md)* interface, implemented for each algorithm. Each supervised model type (eg, `RidgeRegressor`) requires model `fit` and `predict` methods, which are called by the corresponding machine `fit!` and `predict` methods. We don't need to define a  model `predict` method here because MLJ provides a fallback which simply calls the node returned by `fit` on the data supplied: `MLJ.predict(model::Supervised{Node}, Xnew) = yhat(Xnew)`.

Let's now let's wrap our composite model as a tuned model and evaluate on the Boston dataset:

In [33]:
task = load_boston()
X, y = task()
train, test = partition(eachindex(y), 0.7)
wrapped_model = WrappedRidge(ridge_model)

# WrappedRidge @ 1…44: 
ridge_model             =>   RidgeRegressor{Float64} @ 1…83



In [34]:
params(wrapped_model)

(ridge_model = (target_type = Float64, lambda = 0.01),)

In [36]:
nested_ranges = (ridge_model = (lambda = range(ridge_model, :lambda, lower=0.1, upper=100.0, scale=:log10),),)

(ridge_model = (lambda = NumericRange @ 3…60,),)

In [37]:
tuned_wrapped_model = TunedModel(model=wrapped_model, tuning=Grid(resolution=20),
resampling=CV(), measure=rms, nested_ranges=nested_ranges);

In [38]:
tuned_wrapped = machine(tuned_wrapped_model, X, y)
evaluate!(tuned_wrapped, resampling=Holdout(fraction_train=0.7), measure=rms, verbosity=0) |> mean  # nested resampling estimate

6.88236977264247